# **KogSys-ML-B Introduction to Machine Learning**
## **Inductive Logic Programming**
---

To set up a conda environment suitable for this notebook, you can use the following console commands:

```bash
conda create -y -n foil python=3.13
conda activate foil
python -m pip install -r requirements.txt
```

**Note**: Conda can become very hard-drive hungry when you use many environments. Consider regularly deleting environments you no longer need and running the ``conda clean --all`` command to remove no longer needed packages and cached files.

You can also install the requirements for this notebook into an existing environment by running the cell below:

To use ``janus-swi`` you need to have _SWI Prolog_ installed on your system.

The easiest way to do this is via your system's package manager:

- **Windows (chocolatey):** `choco install swi-prolog`
- **macOS (Homebrew):** `brew install swi-prolog`

**Important:** If you install _SWI Prolog_ manually, you may need to add the path to the executable to your system's ``PATH`` variable.

In [4]:
# !python -m pip install -q -U -r requirements.txt

In [ ]:
import contextlib
import os

from copy import copy

import janus_swi as janus

janus.consult("sandwich.pl")

# Disable singleton warnings globally (directive runs on consult)
janus.consult(file="nofile", data=":- style_check(-singleton).")

# Swallow warnings (incl. style warnings) from SWI's message system
janus.consult(file="nofile", data=r"""
:- multifile user:message_hook/3.
:- dynamic   user:message_hook/3.
user:message_hook(_Term, warning, _Lines) :- !.
user:message_hook(_Term, style_warning, _Lines) :- !.
""")

### **Framework Implementation**

In [6]:
class Predicate():
    def __init__(self, head: str, *params: str):
        """
        Parameters
        ----------
        head : str
            predicate head
        *params : str
            an arbitrary amount of parameters as Strings. Make sure to maintain Prolog syntax, i.e. capitalize variables, lowercase constants
        """
        self.head = head
        self.params = params

    def has_variables(self) -> bool:
        """
        Returns ``true`` iff ``self.params`` has at least one variable (string starting with an uppercase letter)
        """
        return any([p[0].isupper() for p in self.params])

    def __repr__(self) -> str:
        return self.head + f"({', '.join(self.params)})" if self.params else self.head


class Example():
    def __init__(self, ex: Predicate, type: bool):
        """
        Parameters
        ----------
        ex : Predicate
            The predicate containing 
        """
        assert not ex.has_variables(), "ex must be a predicate without variables."
        self.__is_positive = type
        self.ex = ex

    @property
    def is_positive(self) -> bool:
        """
        Return is_positive parameter to indicate whether the object is a positive or negative example.
        """
        return self.__is_positive

    def __repr__(self) -> str:
        return self.ex.__repr__()
    

class Rule():
    def __init__(self, trgt: Predicate):
        """
        Parameters
        ----------
        trgt : Predicate
            The target predicate of the rule
        """
        self.trgt = trgt
        self.preconditions = set()

    def add(self, pred: Predicate) -> None:
        """
        Add a precondition to the rule.

        Parameters
        ----------
        pred : Predicate
            The predicate to add as a precondition
        """
        self.preconditions.add(pred)

    def covers(self, ex: Example) -> bool:
        """
        Check if the rule covers the given example.

        Parameters
        ----------
        ex : Example
            The example to check for coverage

        Returns
        -------
        bool
            True if the rule covers the example, False otherwise
        """
        rule_src = (
            f":- style_check(-singleton).\n{self.trgt}."
            if len(self.preconditions) == 0
            else f":- style_check(-singleton).\n{self.trgt} :- {', '.join([pred.__repr__() for pred in self.preconditions])}."
        )
        with open(os.devnull, "w") as devnull:
            with contextlib.redirect_stdout(devnull), contextlib.redirect_stderr(devnull):
                janus.consult(file="nofile", data=rule_src)
                return bool(list(janus.query(f"{ex}.")))
                
    def __repr__(self) -> str:
        return self.trgt.__repr__() if len(self.preconditions) == 0 else f"{self.trgt} :- {', '.join([pred.__repr__() for pred in self.preconditions])}."


def get_examples(trgt: Predicate) -> set[Example]:
    """
    Return a list of all examples in the consulted program file as ``Example`` objects.

    Parameters
    ----------
    trgt: Predicate
        The target predicate, i.e. example structure.

    Returns
    -------
    list[Example]
        A list of all found examples
    """
    pos = list(janus.query(f"pos({trgt})."))    # you can iterate through a query EXACTLY ONCE
    pos = [Example(ex = Predicate(trgt.head, *[sol[param] for param in list(sol.keys())[1:]]), type = True) for sol in pos]

    neg = list(janus.query(f"neg({trgt})."))    # you can iterate through a query EXACTLY ONCE
    neg = [Example(ex = Predicate(trgt.head, *[sol[param] for param in list(sol.keys())[1:]]), type = False) for sol in neg]

    return set(pos + neg)

In [7]:
def FOIL(
    trgt: Predicate,
    preds: set[Predicate],
    exs: set[Example],
    consts: set[str],
    candidate_literals: list[Predicate]
):
    """
    Implements a simple version of the FOIL algorithm, call it "FOILite", if you will ;).

    Parameters
    ----------
    trgt : Predicate
        The target predicate of the rule
    preds : set[Predicate]
        The set of all available predicates
    exs : set[Example]
        The set of all examples
    consts : set[str]
        The set of all constants
    candidate_literals : list[Predicate]
        The list of all candidate literals, sorted in order to be used (this version does not use FOIL-GAIN)

    Returns
    -------
    set[Rule]
        The set of learned rules.
    """
    pos = set([ex for ex in exs if ex.is_positive])
    neg = exs - pos

    learned_rules = set()

    while pos:
        new_rule = Rule(trgt)
        new_rule_neg = copy(neg)
        while new_rule_neg:
            best = candidate_literals.pop(0)
            new_rule.add(best)
            new_rule_neg = set([ex for ex in new_rule_neg if new_rule.covers(ex)])
        learned_rules.add(new_rule)
        pos = set([ex for ex in pos if not new_rule.covers(ex)])

    return learned_rules

In [ ]:
trgt = Predicate("sandwich", "X")

preds = set((
    Predicate("on_block", "X", "Y"),
    Predicate("on_table", "X", "Y"),
    Predicate("free", "X")
))

exs = get_examples(trgt)

consts = set(map(chr, range(ord("a"), ord("g") + 1)))

candidate_literals = [
    Predicate("on_block", "X", "Y"),
    Predicate("on_block", "Z", "X"),
    Predicate("free", "Z"),
]

FOIL(trgt, preds, exs, consts, candidate_literals)

IndexError: pop from empty list